In [1]:
from pathlib import Path
import pandas as pd
from pingouin import ancova
import sys

sys.path.append("../../")
from lib.general import get_stage_list
from lib.r_interface import tukey, tukey_multiple_dvs

Error importing in API mode: ImportError('On Windows, cffi mode "ANY" is only "ABI".')
Trying to import in ABI mode.
Trying to import in ABI mode.
c:\Users\m\Documents\medical-data\.venv\Lib\site-packages\rpy2\rinterface\__init__.py:1211: UserWarning: Environment variable "PATH" redefined by R and overriding existing variable. Current: "c:\Users\m\Documents\medical-data\.venv\Scripts;C:\Users\m\Documents\medical-data\.venv\Scripts;C:\VulkanSDK\1.4.328.1\Bin;C:\Python314\Scripts\;C:\Python314\;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v13.0\bin\x64;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v13.0\bin;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\WINDOWS\System32\OpenSSH\;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files (x86)\Windows Kits\10\Windows Performance Toolkit\;C:\Program Files\NVIDIA Corporation\Nsight Compute 2025.3.0\;C:\Program Files\CMake\bin;C:\Program Files\Git\cmd;C:\Program 

### Input

In [2]:
# Define I/O paths
path_demographics: Path = Path(
    "../../../data/processed/adni/demographics_biomarkers.csv"
).resolve()
path_lipidomics: Path = Path(
    "../../../data/processed/adni/lipidomics_total.csv"
).resolve()
path_lipidomics_dict: Path = Path(
    "../../../data/processed/adni/lipidomics_dict.csv"
).resolve()

In [3]:
# Read files
demographics: pd.DataFrame = pd.read_csv(path_demographics)
lipidomics: pd.DataFrame = pd.read_csv(path_lipidomics).dropna().drop_duplicates()
df: pd.DataFrame = demographics.join(lipidomics.set_index("RID"), on="RID", how="inner")
lipidomics_dict = pd.read_csv(path_lipidomics_dict)

In [4]:
stage_list = get_stage_list(2)

### ANCOVA

In [5]:
# Convert bool columns to int
bool_cols: list[str] = df.select_dtypes(include=[bool]).columns.tolist()
df[bool_cols] = df[bool_cols].astype(int)

In [6]:
# For each lipid class, perform ANCOVA, and store the lipid classes that differ significantly between stages
lipid_differ_between_stages: list[str] = []
for lipid_class in lipidomics_dict["lipid_class"].unique():
    df_ancova: pd.DataFrame = ancova(
        data=df,
        dv=lipid_class,
        between="stage",
        covar=["age", "sex", "bmi", "cog", "apoe4"],
        effsize="np2",
    )
    if df_ancova["p-unc"].values[0] < 0.05:
        lipid_differ_between_stages.append(lipid_class)

In [7]:
lipid_differ_between_stages

['Sphingosine_1_phosphate',
 'Ceramide_1_Phosphate',
 'Monohexosylceramide',
 'Dihexosylceramide',
 'GM1_ganglioside',
 'Sulfatide']

### Tukey HSD post hoc

In [8]:
# Perform Tukey's HSD test for the lipid classes that significantly differ between stages
tukey_result: pd.DataFrame = tukey_multiple_dvs(
    df, ["strem2_log10"] + lipid_differ_between_stages, stage_list
)
# Display only the significant p-values
tukey_result[tukey_result < 0.05]  # .dropna(axis=0, how="all")

R callback write-console: Loading required package: MASS
  
R callback write-console: 
Attaching package: 'TH.data'

  
R callback write-console: The following object is masked from 'package:MASS':

    geyser

  
R callback write-console: 
Attaching package: 'TH.data'

  
R callback write-console: The following object is masked from 'package:MASS':

    geyser

  


,"(CSF-/PET-, CSF+/PET-)","(CSF-/PET-, CSF-/PET+)","(CSF-/PET-, CSF+/PET+)","(CSF+/PET-, CSF-/PET+)","(CSF+/PET-, CSF+/PET+)","(CSF-/PET+, CSF+/PET+)"
strem2_log10,0.004183,NaN,NaN,NaN,NaN,NaN
Sphingosine_1_phosphate,NaN,NaN,0.031611,NaN,NaN,NaN
Ceramide_1_Phosphate,NaN,NaN,NaN,NaN,NaN,NaN
Monohexosylceramide,NaN,NaN,0.025335,NaN,NaN,NaN
Dihexosylceramide,NaN,NaN,0.008511,NaN,0.008670,NaN
GM1_ganglioside,0.047720,NaN,NaN,0.001581,0.003822,NaN
Sulfatide,NaN,NaN,0.035636,NaN,NaN,NaN


In [9]:
# Perform ANCOVA for strem2_log10
ancova(
    data=df,
    dv="strem2_log10",
    between="stage",
    covar=["age", "sex", "cog", "apoe4"],
    effsize="np2",
)

,Source,SS,DF,F,p-unc,np2
0,stage,0.700578,3,4.431890,0.004355,0.026004
1,age,0.714106,1,13.552397,0.000257,0.026493
2,sex,0.045603,1,0.865464,0.352665,0.001735
3,cog,0.025848,1,0.490548,0.484010,0.000984
4,apoe4,0.059457,1,1.128373,0.288637,0.002261
5,Residual,26.240732,498,NaN,NaN,NaN


In [10]:
# Perform Tukey's post-hoc for strem2_log10
tukey(df, "strem2_log10", stage_list)

,coef,lower,upper,std_err,p_adj
"(CSF-/PET-, CSF+/PET-)",0.184211,0.044890,0.323532,0.054753,0.003740
"(CSF-/PET-, CSF-/PET+)",0.036573,-0.061851,0.134996,0.038681,0.767844
"(CSF-/PET-, CSF+/PET+)",0.066080,-0.001939,0.134100,0.026732,0.059712
"(CSF+/PET-, CSF-/PET+)",-0.147638,-0.308353,0.013077,0.063161,0.083757
"(CSF+/PET-, CSF+/PET+)",-0.118131,-0.255962,0.019701,0.054168,0.120142
"(CSF-/PET+, CSF+/PET+)",0.029508,-0.074971,0.133987,0.041060,0.882496
